In [3]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ge

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-12-30 07:11:38--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2020-12-30 07:11:39 (6.61 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [7]:
from pyspark.sql.functions import to_date
from pyspark.sql.types import *
# Read in the Review dataset as a DataFrame
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
votes_20_df = df.filter('total_votes >= 20')
votes_20_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|
|         US|    1243530|R13C5INE1RTZP6|B00G66UDAO|     130471439|Sportspower For B...|            Baby|          2|    

In [9]:
top_50_votes_df = votes_20_df.filter("helpful_votes/total_votes>=.5")
top_50_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|
|         US|    1243530|R13C5INE1RTZP6|B00G66UDAO|     130471439|Sportspower For B...|            Baby|          2|    

In [10]:
vine_y_df = top_50_votes_df.filter('vine == "Y"')
vine_y_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21155796|R3N0XV9267NOXV|B00ZKTVI66|     155258416|Graco LiteRider C...|            Baby|          5|           41|         47|   Y|                N|No more disturbin...|This version of t...| 2015-08-31|
|         US|   25087234| RSA6JQ346JZHZ|B00SIL7EZ2|     916919021|Skip Hop Moonligh...|            Baby|          5|    

In [11]:
vine_n_df = top_50_votes_df.filter('vine == "N"')
vine_n_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|
|         US|    1243530|R13C5INE1RTZP6|B00G66UDAO|     130471439|Sportspower For B...|            Baby|          2|    

In [13]:
#total number of paid reviews
total_paid_reviews=vine_y_df.count()
total_paid_reviews

463

In [29]:
#total number of 5 star paid reviews
total_paid5_reviews=vine_y_df.filter("star_rating==5").count()
total_paid5_reviews

202

In [30]:
#percentage of 5 star paid reviews
perc_paid5_reviews= (total_paid5_reviews / total_paid_reviews)*100
perc_paid5_reviews

43.628509719222464

In [31]:
#total number of unpaid reviews
total_unpaid_reviews=vine_n_df.count()
total_unpaid_reviews

25094

In [32]:
#total number of 5 star unpaid reviews
total_unpaid5_reviews=vine_n_df.filter("star_rating==5").count()
total_unpaid5_reviews

12033

In [33]:
#percentage of 5 star unpaid reviews
perc_unpaid5_reviews= (total_unpaid5_reviews / total_unpaid_reviews)*100
perc_unpaid5_reviews

47.95170160197657

In [42]:
# Create DataFrame with results 
dataframe = spark.createDataFrame([
                                   ("Paid Vine", total_paid_reviews, total_paid5_reviews, perc_paid5_reviews),
                                   ("Unpaid Vine", total_unpaid_reviews, total_unpaid5_reviews, perc_unpaid5_reviews),
], ["Reviews", "Total Reviews", "Total 5-star Reviews", "Percentage of 5-star Reviews"])

dataframe.show()

+-----------+-------------+--------------------+----------------------------+
|    Reviews|Total Reviews|Total 5-star Reviews|Percentage of 5-star Reviews|
+-----------+-------------+--------------------+----------------------------+
|  Paid Vine|          463|                 202|          43.628509719222464|
|Unpaid Vine|        25094|               12033|           47.95170160197657|
+-----------+-------------+--------------------+----------------------------+



In [43]:

# Convert PySpark DataFrame to Pandas DataFrame
import pandas as pd
pandas_df = dataframe.toPandas()
pandas_df

,Reviews,Total Reviews,Total 5-star Reviews,Percentage of 5-star Reviews
0,Paid Vine,463,202,43.628510
1,Unpaid Vine,25094,12033,47.951702


In [44]:
pandas_df=pandas_df.style.hide_index()
pandas_df

Reviews,Total Reviews,Total 5-star Reviews,Percentage of 5-star Reviews
Paid Vine,463,202,43.628510
Unpaid Vine,25094,12033,47.951702
